<a href="https://colab.research.google.com/github/shahchayan9/Timegpt-Tabula9-and-Relational-Deep-Learning/blob/main/RDL_and_relbench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, install the required dependencies
!pip install torch
!pip install torch_geometric
!pip install relbench

# Install additional dependencies if needed
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv pyg_lib -f https://data.pyg.org/whl/torch-2.0.0+cu117.html


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 999.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 1.3 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.2/887.2 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.6 MB/s eta 0:00:00


In [ ]:
import relbench

relbench.__version__

'1.1.0'

In [ ]:
# Import necessary libraries
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
print(f"Dataset: {dataset}, Number of graphs: {len(dataset)}")

# Move data to the specified device
data = dataset[0].to(device)


Using device: cpu


Processing...


Dataset: Cora(), Number of graphs: 1


Done!


In [ ]:
# Define a GNN model for node classification
class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize model, loss function, and optimizer
model = GNNModel(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [ ]:
# Training function
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluation function
@torch.no_grad()
def evaluate():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)

    # Calculate accuracy
    train_acc = accuracy_score(data.y[data.train_mask].cpu(), pred[data.train_mask].cpu())
    val_acc = accuracy_score(data.y[data.val_mask].cpu(), pred[data.val_mask].cpu())
    test_acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())

    return train_acc, val_acc, test_acc


In [ ]:
# Training loop
epochs = 200
for epoch in range(1, epochs + 1):
    loss = train()
    train_acc, val_acc, test_acc = evaluate()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:02d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")


Epoch 10, Loss: 0.0014, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7800
Epoch 20, Loss: 0.0012, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7800
Epoch 30, Loss: 0.0011, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7800
Epoch 40, Loss: 0.0010, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7790
Epoch 50, Loss: 0.0009, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7790
Epoch 60, Loss: 0.0009, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7790
Epoch 70, Loss: 0.0008, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7790
Epoch 80, Loss: 0.0007, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7790
Epoch 90, Loss: 0.0007, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7800
Epoch 100, Loss: 0.0006, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7790
Epoch 110, Loss: 0.0006, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7800
Epoch 120, Loss: 0.0006, Train Acc: 1.0000, Val Acc: 0.7700, Test Acc: 0.7790
Epoch 130, Loss: 0.0005, Train Acc: 1.0000, Val Acc: 0.7680, Test Acc: 0.

In [ ]:
_, _, test_acc = evaluate()
print(f"Final Test Accuracy: {test_acc:.4f}")


Final Test Accuracy: 0.7800
